# 1. References

Title: kmeans text clustering

Link: https://pythonprogramminglanguage.com/kmeans-text-clustering/

Link: https://www.experfy.com/blog/k-means-clustering-in-text-data


# 2. Imports

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.tokenize import ToktokTokenizer

# 3. Data cleaning

In [43]:
# file_name = 'TED_Talks_by_ID_plus-transcripts-and-LIWC-and-MFT-plus-views.csv'
file_name = 'TED_Transcripts_short.csv'
df = pd.read_csv('../owentemple-ted-talks-complete-list/{}'.format(file_name))
df.head()

,id,speaker,headline,description,tags,transcript
0,1,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ..."
1,2,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ..."
2,3,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r..."
3,4,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...","aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst..."
4,5,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac..."


In [3]:
df.iloc[:,:15].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 6 columns):
id             11 non-null int64
speaker        11 non-null object
headline       11 non-null object
description    11 non-null object
tags           11 non-null object
transcript     11 non-null object
dtypes: int64(1), object(5)
memory usage: 608.0+ bytes


In [4]:
df = df.dropna(subset=['transcript'])
df = df.reset_index(drop=True)
df.iloc[:,:15].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 6 columns):
id             11 non-null int64
speaker        11 non-null object
headline       11 non-null object
description    11 non-null object
tags           11 non-null object
transcript     11 non-null object
dtypes: int64(1), object(5)
memory usage: 608.0+ bytes


In [36]:
df['transcript'][8]

'0:11\r\r\rAs you pointed out, every time you come here, you learn something.\rThis morning, the world\'s experts\rfrom I guess three or four different companies on building seats,\rI think concluded that ultimately, the solution is, people shouldn\'t sit down.\rI could have told them that.\r(Laughter)\rYesterday, the automotive guys gave us some new insights.\rThey pointed out that, I believe it was between 30 and 50 years from today,\rthey will be steering cars by wire,\rwithout all that mechanical stuff.\r(Laughter)\rThat\'s reassuring.\r(Applause)\rThey then pointed out that there\'d be, sort of, the other controls by wire,\rto get rid of all that mechanical stuff.\rThat\'s pretty good, but why not get rid of the wires?\rThen you don\'t need anything to control the car, except thinking about it.\rI would love to talk about the technology,\rand sometime, in what\'s past the 15 minutes,\rI\'ll be happy to talk to all the techno-geeks around here\rabout what\'s in here.\rBut if I had 

In [44]:
df['transcript'] = df['transcript'].str.replace('\r',' ')
df['transcript'] = df['transcript'].str.replace("\'s"," is")
df['transcript'] = df['transcript'].str.replace("\'m"," am")
df['transcript'] = df['transcript'].str.replace("\'ll"," will")
df['transcript'] = df['transcript'].str.replace("n\'t"," not")
df['transcript'] = df['transcript'].str.replace("\'ve"," have")
df['transcript'] = df['transcript'].str.replace("\'re"," are")
df['transcript'] = df['transcript'].str.replace("\'d"," would")
df['transcript'] = df['transcript'].str.replace(r"\(([^)]+)\)","")
# Deal with Mr. and Dr.
df['transcript'] = df['transcript'].str.replace("mr.","mr")
df['transcript'] = df['transcript'].str.replace("Mr.","mr")
df['transcript'] = df['transcript'].str.replace("dr.","dr")
df['transcript'] = df['transcript'].str.replace("mrs.","mrs")
df['transcript'] = df['transcript'].str.replace("Mrs.","mrs")
df['transcript'] = df['transcript'].str.replace("Dr.","dr")

In [45]:
import string
df['transcript'] = df['transcript'].str.replace(r'\d+','')
df['transcript'] = df['transcript'].str.replace(r'<.*?>','')
for i in string.punctuation:
    if i == "'":
        df['transcript'] = df['transcript'].str.replace(i,'')
    elif i == ".":
        continue
    else:
        df['transcript'] = df['transcript'].str.replace(i,' ')
df['transcript'] = df['transcript'].str.replace('\s+',' ')

In [46]:
df['transcript'][8]

' As you pointed out every time you come here you learn something. This morning the world is experts from I guess three or four different companies on building seats I think concluded that ultimately the solution is people should not sit down. I could have told them that. Yesterday the automotive guys gave us some new insights. They pointed out that I believe it was between and years from today they will be steering cars by wire without all that mechanical stuff. That is reassuring. They then pointed out that there would be sort of the other controls by wire to get rid of all that mechanical stuff. That is pretty good but why not get rid of the wires Then you do not need anything to control the car except thinking about it. I would love to talk about the technology and sometime in what is past the minutes I will be happy to talk to all the techno geeks around here about what is in here. But if I had one thing to say about this before we get to first it would be that from the time we st

In [47]:
token=ToktokTokenizer()
stopWordList=stopwords.words('english')


def stopWordsRemove(text):
    wordList=[x.lower().strip() for x in token.tokenize(text)]
    removedList=[x + ' ' for x in wordList if not x in stopWordList]
    text=''.join(removedList)
    return text
df['transcript'] = df['transcript'].map(lambda com : stopWordsRemove(com))

In [48]:
print(df['transcript'][8])

pointed every time come learn something. morning world experts guess three four different companies building seats think concluded ultimately solution people sit down. could told that. yesterday automotive guys gave us new insights. pointed believe years today steering cars wire without mechanical stuff. reassuring. pointed would sort controls wire get rid mechanical stuff. pretty good get rid wires need anything control car except thinking it. would love talk technology sometime past minutes happy talk techno geeks around here. one thing say get first would time started building big idea technology. really big idea technology started applying ibot disabled community. big idea think new piece solution fairly big problem transportation. maybe put perspective much data happy give different forms. never know strikes fancy everybody perfectly willing believe car changed world. henry ford years ago started cranking model ts. think people think context technology applied. instance time perce

## 3.1 Split into sentences and then paragraphs

In [65]:
def split_text_paragraphs(text, merge_amount):
    ls = text.split('.')
    paras = []
    sample = ''
    for index, value in enumerate(ls):
        sample += value
        if index % merge_amount == 0 and index/merge_amount >= 1:
            paras.append(sample)
            sample = ''
        elif len(ls) == index:
            paras.append(sample)
    return paras
sample = split_text_paragraphs(df['transcript'][8], 10)
print(sample)
print(len(sample))

['pointed every time come learn something morning world experts guess three four different companies building seats think concluded ultimately solution people sit down could told that yesterday automotive guys gave us new insights pointed believe years today steering cars wire without mechanical stuff reassuring pointed would sort controls wire get rid mechanical stuff pretty good get rid wires need anything control car except thinking it would love talk technology sometime past minutes happy talk techno geeks around here one thing say get first would time started building big idea technology really big idea technology started applying ibot disabled community', ' big idea think new piece solution fairly big problem transportation maybe put perspective much data happy give different forms never know strikes fancy everybody perfectly willing believe car changed world henry ford years ago started cranking model ts think people think context technology applied instance time percent america

# 4. Perform K Means

In [66]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(sample)
print(df['tags'][8])
'''
robots,cars,industrial design,transportation,invention,education,innovation,social change,science,sustainability,technology,business

transport: cars
industrial design:
innovation: invention, technology, robots, science
education:
sustainability:
social change:
business: 
'''

robots,cars,industrial design,transportation,invention,education,innovation,social change,science,sustainability,technology,business


In [73]:
true_k = 7
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
# print(order_centroids)
terms = vectorizer.get_feature_names()
# print(terms)
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind] + ', ', end = ''),
    print("\n")

print("\n")
# print("Prediction")

Y = vectorizer.transform(["stirling cycle engine confused lot things doing little beast right producing hundrd watts electricity yes could attached yes kilogram propane could drve new york boston choose perhaps interesting little engine burn fuel might skeptical capability impact world ca simply plug volt outlet working actually alternative energy source starting way back johnson johnson run ibot best batteries could get watt hours per kilogram lead watt hours per kilogram nickel cadmium watt hours per kilogram nickel metal hydrde watt hours per kilogram lithium watt hours energy every kilogram propane gasoline nobody drves electric cars"])
prediction = model.predict(Y)
print(prediction)

# Y = vectorizer.transform(["My cat is hungry."])
# prediction = model.predict(Y)
# print(prediction)

Top terms per cluster:
Cluster 0:
 says,  looks,  little,  zipping,  going,  technology,  ca,  things,  lady,  camera, 

Cluster 1:
 people,  farm,  efficient,  cities,  think,  point,  view,  world,  city,  high, 

Cluster 2:
 water,  kilogram,  electricity,  heat,  hours,  watts,  watt,  burn,  little,  waste, 

Cluster 3:
 cities,  minutes,  make,  got,  start,  car,  walk,  took,  cars,  times, 

Cluster 4:
 big,  carriage,  horse,  pointed,  car,  idea,  technology,  started,  rid,  wire, 

Cluster 5:
 sustainable,  fighting,  need,  population,  right,  fun,  percent,  way,  complicated,  middle, 

Cluster 6:
 belong,  pedestrian,  vehicle,  zero,  seriously,  street,  concerned,  greyhound,  ground,  general, 



[2]
